In [ ]:
!pip install openai
import openai

# 발급받은 API 키 설정
OPENAI_API_KEY = ""

# openai API 키 인증
openai.api_key = OPENAI_API_KEY

In [ ]:
from google.colab import drive
import pandas as pd
import openai
from google.colab import drive
import os
from openai import OpenAI
import requests

# OpenAI API 키 설정
OPENAI_API_KEY = OPENAI_API_KEY  # 사용자의 API 키로 대체해야 함
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# OpenAI 클라이언트 생성
client = OpenAI(api_key=OPENAI_API_KEY)

# 구글 드라이브 마운트
drive.mount('/content/drive')

# Excel 파일 경로 설정
excel_file_path = '/content/drive/My Drive/dangam/dataset/test_crawling.xlsx'

# Excel 파일 읽기
df = pd.read_excel(excel_file_path)

# 주어진 사자성어를 ChatGPT에게 긍정적인 해석과 부정적인 해석을 물어보는 함수
def ask_gpt_for_proverb(proverb):
    # ChatGPT를 사용한 텍스트 분류 요청
    positive_response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": f"'{proverb}' 이 문장을 어려운 사람들에게 힘을 주는 말로 만들어줘."}],
    )
    negative_response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content":f"'{proverb}'이 문장을 경고하는 말로 만들어줘."}],
    )

    # 응답에서 긍정적, 부정적 해석 추출
    positive_interpretation = positive_response.choices[0].message.content
    negative_interpretation = negative_response.choices[0].message.content

    return positive_interpretation, negative_interpretation

# 주요 프로세스
def main():
    positive_sentiments = []
    negative_sentiments = []

    # 모든 사자성어에 대해 ChatGPT에게 긍정적인 해석과 부정적인 해석을 물어봄
    for proverb in df['contents_detail']:
        positive_sentiment, negative_sentiment = ask_gpt_for_proverb(proverb)
        positive_sentiments.append(positive_sentiment)
        negative_sentiments.append(negative_sentiment)

    # 새로운 DataFrame 생성
    new_df = pd.DataFrame({
        'contents_kr': df['contents_kr'],
        'positive_sentiment': positive_sentiments,
        'negative_sentiment': negative_sentiments
    })

    # 새로운 Excel 파일로 저장
    new_excel_file_path = '/content/drive/My Drive/dangam/dataset/test_crawling_new_3.xlsx'
    new_df.to_excel(new_excel_file_path, index=False)

if __name__ == "__main__":
    main()